In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [12]:
env = gym.make('MountainCar-v0')

/opt/conda/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [54]:
img = Image.new('L',(10,10),color=128)

In [7]:
img.putpixel((5,5),255)

In [55]:
img

In [27]:
def probecrash(x,y):
    observation = env.reset()
    env.env.state = np.array([x, y])
    first = 200
    for t in range(333):
        #env.render()
        #print(observation, action)
        observation, reward, done, info = env.step(2)
        #print(done)
        if done:
            first = t
            break
    env.env.state = np.array([x, y])
    ac = 0
    for t in range(333):
        #env.render()
        #print(observation, action)
        if observation[1] == 0:
            ac = 2
        observation, reward, done, info = env.step(ac)
        #print(done)
        if done:
            return t <= first            
    print("Error: left loop")
    
def compare(x,y):
    if x<y:
        return 0#, x, y
    if x>y:
        return 255
    return 128
    
def probeol(x,y):
    observation = env.reset()
    env.env.state = np.array([x, y])
    first = 200
    for t in range(333):
        #env.render()
        #print(observation, action)
        observation, reward, done, info = env.step(2)
        #print(done)
        if done:
            first = t
            break
    observation = env.reset()
    env.env.state = np.array([x, y])
    for t in range(333):
        #env.render()
        #print(observation, action)
        observation, reward, done, info = env.step(0 if t==0 else 2)
        #print(done)
        if done:
            return compare(t, first)       
    print("Error: left loop")

In [70]:
def dopol(obs, pol):
    px = int((obs[0]-env.min_position)/(env.max_position - env.min_position) * 1023.9999)
    py = int((obs[1]+env.max_speed)/(2*env.max_speed) * 1023.9999)
    return 2 if pol.getpixel((px,py)) > 200 else 0

In [32]:
res = 256
imgol = Image.new('L',(res,res))


stepx = (env.max_position-env.min_position)/res
startx = env.min_position + stepx/2

stepy = 2*env.max_speed/res
starty = -env.max_speed + stepy/2

p = 1

for i in range(res):
    x = startx + i*stepx
    if i == ((p*res)//10):
        print('+')
        p = p+1
    for k in range(res):
        y = starty + k*stepy
        imgol.putpixel((i,k), probeol(x,y))


+
+
+
+
+
+
+
+
+


In [44]:
mask1 = Image.open('mask1.png').convert('L')

In [59]:
res = 1024
imgol2 = Image.new('L',(res,res),color=50)

stepx = (env.max_position-env.min_position)/res
startx = env.min_position + stepx/2

stepy = 2*env.max_speed/res
starty = -env.max_speed + stepy/2

p = 1

for i in range(res):
    x = startx + i*stepx
    if i == ((p*res)//10):
        print('+')
        p = p+1
    for k in range(res):
        y = starty + k*stepy
        if mask1.getpixel((i,k)) > 200:
            imgol2.putpixel((i,k), probeol(x,y))

+
+
+
+
+
+
+
+
+


In [61]:
pol1 = Image.open('policy1.png').convert('L')

In [87]:
def probeor(x,y):
    env.reset()
    observation = np.array([x, y])
    env.env.state = np.array([x, y])
    first = 200
    for t in range(333):
        #env.render()
        action = dopol(observation, pol1)
        #print(observation, action)
        observation, reward, done, info = env.step(action)
        #print(done)
        if done:
            first = t
            break
    env.reset()
    observation = np.array([x, y])
    env.env.state = np.array([x, y])
    for t in range(333):
        #env.render()
        action = 2 if t==0 else dopol(observation, pol1)
        #print(observation, action)
        observation, reward, done, info = env.step(action)
        #print(done)
        if done:
            #print(t, first)
            return compare(t, first)       
    print("Error: left loop")

In [89]:
res = 256
imgor = Image.new('L',(res,res))


stepx = (env.max_position-env.min_position)/res
startx = env.min_position + stepx/2

stepy = 2*env.max_speed/res
starty = -env.max_speed + stepy/2

p = 1

for i in range(res):
    x = startx + i*stepx
    if i == ((p*res)//10):
        print('+')
        p = p+1
    for k in range(res):
        y = starty + k*stepy
        imgor.putpixel((i,k), probeor(x,y))

+
+
+
+
+
+
+
+
+


In [91]:
mask2 = Image.open('mask2.png').convert('L')

In [94]:
res = 1024
imgor2 = Image.new('L',(res,res),color=50)

stepx = (env.max_position-env.min_position)/res
startx = env.min_position + stepx/2

stepy = 2*env.max_speed/res
starty = -env.max_speed + stepy/2

p = 1

for i in range(res):
    x = startx + i*stepx
    if i == ((p*res)//10):
        print('+')
        p = p+1
    for k in range(res):
        y = starty + k*stepy
        if mask2.getpixel((i,k)) > 200:
            imgor2.putpixel((i,k), probeor(x,y))

+
+
+
+
+
+
+
+
+


In [96]:
pol2 = Image.open('policy2.png').convert('L')

In [1]:
import gym
import sys
sys.path.append('../../')
from Code.envs.MountainCar import LookupPolicy
import torch

In [2]:
pred = LookupPolicy()
env = gym.make('MountainCar-v0')


/opt/conda/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
while True:
    obs = env.reset()
    for i in range(200):
        a = pred(torch.tensor(obs).unsqueeze(0).unsqueeze(0))#dopol(obs, pol2) #0 if i < 45 else 2
        #print(obs.shape, a.shape, a)
        obs, _, t, _ = env.step(int(a.item()))
        env.render()
        if t:
            print(i+1)
            break

103
103
105
102
103
103
100
102
103
88
103
103
84
103
87
94
104
103


KeyboardInterrupt: 